openai function with LCEL. Pydantic python library which makes it easier to construct openai functions

Pydanytic

Pydantic is data validation library for python

1. Works with python type annotations. But rather than static type checking, they are actively used at runtime for data validation and conversion

2. Provides built-in methods to serialize/deserialize models to/from JSON, dictionaries etc

3. Langchain leverages Pydantic to create JSON Scheme describing function.

In normal python you would create a class like this:

class user:

    def __init__(self, name:str, age:int, email:str):

        self.name = name

        self.age = age

        self.email = email



using Pydantic:-

from pydantic import BaseModel,

class User(Basemodel):

        name: str

        age:int

        email:Emailstr

In [1]:
import os
from dotenv import load_dotenv
import openai

In [3]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 7


In [4]:
from typing import List
from pydantic import BaseModel, Field

In [5]:
#create normal python class


class User:
    def __init__(self,name:str, age:int, email:str):
        self.name = name
        self.age = age
        self.email = email

In [9]:
foo = User(name="Joe",age = 'bar', email="joe@gmail.com")

In [10]:
foo.name

'Joe'

suppose we assign age value to bar, will print bar as output which is wrong. as we want some validation say its integer

In [12]:
foo.age

'bar'

In [13]:
class pUser(BaseModel):
    name:str
    age:int
    email:str

In [16]:
foo_p = pUser(name="Joe",age = 32, email="joe@gmail.com")

In [17]:
foo_p

pUser(name='Joe', age=32, email='joe@gmail.com')

lets try with pydantic try invalid argument for age like before.it does validation

In [18]:
foo_p = pUser(name="Joe",age = 'bar', email="joe@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing

In [19]:
class Class(BaseModel):
    students:List[pUser]

In [20]:
obj = Class(
    students= [pUser(name="joe",age=32, email="joe@gmail.com")]
)

In [21]:
obj

Class(students=[pUser(name='joe', age=32, email='joe@gmail.com')])

Pydanic to create OpenAI function definations

In [23]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code:str = Field(description="airport code to get weather for")  # Field pydantic module used to set things of various fields.

In [24]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [25]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_1910/1313296718.py:1: LangChainDeprecationWarning: The function `_convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  weather_function = convert_pydantic_to_openai_function(WeatherSearch)


In [26]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [30]:
class WeatherSearch1(BaseModel):
    airport_code:str = Field(description="airport code to get weather for")  # Field pydantic module used to set things of various fields.

In [31]:
weather_function1 = convert_pydantic_to_openai_function(WeatherSearch1)

In [32]:
weather_function1

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [37]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code:str
    
    #descriptions for arguments are optional in Langchain. 

In [35]:
weather_function2 = convert_pydantic_to_openai_function(WeatherSearch2)

In [36]:
weather_function2

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [38]:
from langchain.chat_models import ChatOpenAI

In [39]:
model = ChatOpenAI()

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_1910/1603725051.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()


In [40]:
model.invoke("what is the weather in SF today?",functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 70, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--ea74c70a-dda6-435c-b7fe-1229f9406bd4-0')

In [41]:
model_with_function = model.bind(functions=[weather_function])

In [42]:
model_with_function.invoke("What is the weather in SF?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 69, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--c2a878f2-667b-4ba7-af3f-f7c201a3d207-0')

In [43]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [ ]:
model_with_forced_function.invoke("What is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 79, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--90eb539b-98d3-4184-b43f-3d42c0d3d33b-0')

In [45]:
model_with_forced_function.invoke("Hii")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 74, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--602d15ee-f6d6-44f7-9caf-11b4bd30a4cd-0')

here we can see its still forcing up the function to call even though the query is invalid.

Using in a chain

We can use this model bound to function in a chain as we normally world

In [46]:
from langchain.prompts import ChatPromptTemplate

In [49]:
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant"),
    ("user", "{input}")
    ])

In [50]:
chain = prompt | model_with_function

In [51]:
chain.invoke({"input":"What is the weather in SF?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 75, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--27993aa1-a4f3-46f8-a31f-61f81705e6ea-0')

Using Multiple Functions

Even better, we can pass a set of functions and let the LLM decide which to use based on the question context.

In [53]:
class ArtistSearch(BaseModel):
    """ Call this to get the name of songs by a particular artist"""
    artist_name: str = Field(description="name of the artist to look up")
    n: int = Field(description="number of results")

In [54]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [55]:
model_with_functions = model.bind(functions=functions)

In [56]:
model_with_functions.invoke("What is the weather in SF?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 108, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--33c77908-5476-48f6-b68f-6cbf9d56347d-0')

In [57]:
model_with_functions.invoke("What are three songs by Taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":3}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 109, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--7d8378e8-9af1-4586-b44d-6aac42be4d16-0')

In [58]:
model_with_functions.invoke("Hii!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 104, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--34f7eeb3-d397-4487-a4ce-b3db3ec3c954-0')